The Wikipedia Preview ETL job now an old Oozie version, which is producing data to `wmf_product.wikipediapreview_stats`, and a new Airflow version, which is producing data to `wmf_product.wikipedia_preview_stats` ([T333218](https://phabricator.wikimedia.org/T333218)). Are there any issues that block retiring the old one in favor of the new one?

In [1]:
import pandas as pd
import wmfdata as wmf

from wmfdata.utils import (
    pd_display_all
)

In [2]:
print(wmf.presto.run("""
    SHOW CREATE TABLE wmf_product.wikipediapreview_stats
""").iloc[0, 0])

CREATE TABLE analytics_hive.wmf_product.wikipediapreview_stats (
   "pageviews" bigint COMMENT 'Number of pageviews shown as a result of a clickthrough from a Wikipedia Preview preview',
   "previews" bigint COMMENT 'Number of API requests for article preview content made by Wikipedia Preview clients',
   "year" integer COMMENT 'Unpadded year of request',
   "month" integer COMMENT 'Unpadded month of request',
   "day" integer COMMENT 'Unpadded day of request',
   "device_type" varchar COMMENT 'Type of device used by the client: touch or non-touch',
   "referer_host" varchar COMMENT 'Host from referer parsing',
   "continent" varchar COMMENT 'Continent of the accessing agents (maxmind GeoIP database)',
   "country_code" varchar COMMENT 'Country iso code of the accessing agents (maxmind GeoIP database)',
   "country" varchar COMMENT 'Country (text) of the accessing agents (maxmind GeoIP database)',
   "instrumentation_version" integer COMMENT 'Version number incremented along with major

In [5]:
old = wmf.presto.run("""
    SELECT
        year,
        month,
        day,
        referer_host AS website,
        device_type,
        country_code,
        instrumentation_version,
        pageviews,
        previews
    FROM wmf_product.wikipediapreview_stats
    WHERE
        year = 2023
        AND month = 6
    ORDER BY
        year,
        month,
        day,
        referer_host,
        device_type,
        continent,
        country_code,
        instrumentation_version
""")

In [7]:
print(wmf.presto.run("""
    SHOW CREATE TABLE wmf_product.wikipedia_preview_stats
""").iloc[0, 0])

CREATE TABLE analytics_hive.wmf_product.wikipedia_preview_stats (
   "pageviews" bigint COMMENT 'Number of pageviews shown as a result of a clickthrough from a Wikipedia Preview preview',
   "previews" bigint COMMENT 'Number of API requests for article preview content made by Wikipedia Preview clients',
   "device_type" varchar COMMENT 'Type of device used by the client: touch or non-touch',
   "referer_host" varchar COMMENT 'Host from referer parsing',
   "continent" varchar COMMENT 'Continent of the accessing agents (maxmind GeoIP database)',
   "country_code" varchar COMMENT 'Country iso code of the accessing agents (maxmind GeoIP database)',
   "country" varchar COMMENT 'Country (text) of the accessing agents (maxmind GeoIP database)',
   "instrumentation_version" integer COMMENT 'Version number incremented along with major instrumentation changes',
   "year" integer COMMENT 'Unpadded year of request',
   "month" integer COMMENT 'Unpadded month of request',
   "day" integer COMMENT

In [8]:
new = wmf.presto.run("""
    SELECT
        year,
        month,
        day,
        referer_host AS website,
        device_type,
        country_code,
        instrumentation_version,
        pageviews,
        previews
    FROM wmf_product.wikipedia_preview_stats
    WHERE
        year = 2023
        AND month = 6
    ORDER BY
        year,
        month,
        day,
        referer_host,
        device_type,
        continent,
        country_code,
        instrumentation_version
""")

In [9]:
new.equals(old)

True

Well, that was easy! Yes, we can switch.